# Callbacks

Every tool in the `mpltoolbox` comes with entry points for adding callbacks to different events.
The different events are:

- `on_create`: called when drawing the shape (rectangle, line or polygon) is complete
- `on_remove`: called when the shape is deleted (middle click)
- `on_vertex_press`: called when a vertex is clicked (left click)
- `on_vertex_move`: called when a vertex is moved
- `on_vertex_release`: called when the mouse button is released after clicking a vertex
- `on_drag_press`: called when the entire shape (rectangle, line, etc..) is right clicked to initiate drag
- `on_drag_move`: called for every movement during shape drag (right click and hold)
- `on_drag_release`: called when the shape is released after drag

Below is a couple of examples on how these callbacks are used.

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
plt.ioff()
import mpltoolbox as tbx

## Example 1: Add markers to slice 3d cube

We first make some three-dimensional data:

In [ ]:
N = 200
M = 300
L = 100
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
zz = np.arange(L, dtype=np.float64)
x, y, z = np.meshgrid(xx, yy, zz, indexing='ij')
b = N/20.0
c = M/2.0
d = L/2.0
r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2 + ((z-d)/b)**2)
a = 10.0 * np.sin(r) + np.random.random([N, M, L])

Create a figure to display the first `z` slice of the data as a two-dimensional image,
as well as an empty subplot below:

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(7, 7))
fig.canvas.header_visible = False
fig.tight_layout()
ax[0].imshow(a[..., 0], interpolation='none', origin='lower')

Then we add a `Points` tool where:

- When a dot is added on the image, a line is created in the lower panel, showing a one-dimensional `z` slice at the location of the marker
- When a dot is moved, the `z` line is updated accordingly

In [ ]:
points = tbx.Points(ax=ax[0], mec='white')

def make_line(change):
    event = change['event']
    ix = int(event.xdata)
    iy = int(event.ydata)
    l, = ax[1].plot(a[iy, ix, :])
    change['artist'].associated = l

def update_line(change):
    event = change['event']
    ix = int(event.xdata)
    iy = int(event.ydata)
    change['artist'].associated.set_ydata(a[iy, ix, :])

points.on_create = make_line
points.on_vertex_move = update_line

In [ ]:
from dataclasses import dataclass
@dataclass
class Event:
    button = 1
    inaxes = ax[0]
    xdata: float
    ydata: float

points._on_button_press(Event(xdata=101, ydata=67))
points._on_button_press(Event(xdata=116, ydata=96))
points._on_button_press(Event(xdata=271, ydata=155))

In [ ]:
fig

## Example 2: Histogram inside a rectangular region

In the second example, we use the `Ractangles` tool to draw rectangles on the same 2d image.
This defines a region of interest, inside which the data will be histogrammed and displayed on the lower panel. 

In [ ]:
fig2, ax2 = plt.subplots(2, 1, figsize=(7, 7))
fig2.canvas.header_visible = False
fig2.tight_layout()
ax2[0].imshow(a[..., 0], interpolation='none', origin='lower')

In [ ]:
rects = tbx.Rectangles(ax=ax2[0], facecolor=(0, 0, 0, 0.3))

def make_hist(change):
    r = change['artist']
    xy = r.get_xy()
    ix0 = int(xy[0])
    iy0 = int(xy[1])
    ix1 = int(xy[0] + r.get_width())
    iy1 = int(xy[1] + r.get_height())
    n, bins, patches = ax2[1].hist(
        a[min(iy0, iy1):max(iy0, iy1), min(ix0, ix1):max(ix0, ix1), :].ravel(),
        edgecolor=r.get_edgecolor(), facecolor='None')
    r.associated = patches

def update_hist(change):
    change['artist'].associated.remove()
    make_hist(change=change)

def update_vertices(change):
    change['artist'] = change['artist']._patch
    update_hist(change)

rects.on_create = make_hist
rects.on_drag_move = update_hist
rects.on_vertex_move = update_vertices

In [ ]:
from dataclasses import dataclass
@dataclass
class Event:
    button = 1
    inaxes = ax2[0]
    xdata: float
    ydata: float

rects._on_button_press(Event(xdata=51, ydata=67))
ev = Event(xdata=159, ydata=125)
rects._resize_patch(ev)
rects._persist_patch(ev)

rects._on_button_press(Event(xdata=200, ydata=130))
ev = Event(xdata=260, ydata=175)
rects._resize_patch(ev)
rects._persist_patch(ev)

In [ ]:
fig2